CódigraphTesto disponível [na minha página do Github](https://github.com/arthurkenzo/atividades_ia525)

### Importando bibliotecas

In [3]:
import cvxpy as cp
import numpy as np
import mosek
import matplotlib.pyplot as plt
import time
import networkx as nx

from typing import Tuple
from itertools import product

## Questão 1: Pintar o tabuleiro

### Modelo

## Questão 2: Sudoku

### Modelo:

\begin{align*}
    \text{Minimizar }   & ijk x_{ijk}\\
    \text{sujeito a }   & \sum_{k} x_{ijk} = 1 \quad \forall  i,j  \hspace{20pt} \text{(Alocar apenas um número a cada casa)} \\
                        & \sum_{i} x_{ijk} = 1 \quad \forall  j,k  \hspace{20pt} \text{(Unicidade de cada número em cada coluna)} \\
                        & \sum_{j} x_{ijk} = 1 \quad \forall  i,k  \hspace{20pt} \text{(Unicidade de cada número em cada linha)} \\
                        & \sum_{p,p',q,q'} x_{(3p+q),(3p'+q'),k} = 1 \quad \forall  k;  \quad p,p',q,q' \in \{0,1,2\} \hspace{20pt} \text{(Unicidade de cada número em cada submatriz)} \\
                        & x_{ijk} = 1 \quad \forall i,j,k: \: \text{board}(i,j) = k \hspace{20pt} \text{(Fixar valores do tabuleiro inicial)}\\
                        & x_{ijk}\in \mathbb{B} \quad \forall  i,j,k  \hspace{20pt} \text{(Variáveis de decisão binárias)} \\
\end{align*}

onde os índices $(i,j,k)$ que indexam as variáveis são números interios no intervalo $[0,8]$, e as variáveis de decisão $x_{ijk} \in \mathbb{B}$ são não nulas $\iff$ $k$ for o número escolhido para a casa $(i,j)$ do tabuleiro.  Assumimos que o tabuleiro inicial é dado por uma matriz $\text{board}$ cujas casas vazias são preenchidas com zero.

Como o problema pode ser altamente degenerado (por se tratar de um problema de factibilidade), definimos um ordenamento arbitrário das soluções através da função objetivo.

In [31]:
def SolveSudoku(board:np.ndarray) -> np.ndarray:
    """ Solves a sudoku board by building and solving a linear program. 

    Args:
        board (np.ndarray): An array with the initial board values. Empty cells are assumed to be zero-valued.

    Returns:
        np.ndarray: A 3d array containing the optimal values of the model's binary decision variables.
    """

    boardSize = len(board[:,0])
    submatrixSize = 3
    digits = range(boardSize)
    submatrices = range(submatrixSize)

    # decision variables
    decisionVars = np.empty((boardSize, boardSize, boardSize), dtype=object)
    for i in digits:
        for j in digits:
            for k in digits:
                decisionVars[i,j,k] = cp.Variable(boolean=True, name=f"x_{i}{j}{k}")

    # constraints
    constraints = []
    # one single value per cell
    for i in digits:
        for j in digits:
            constraints += [np.sum(decisionVars[i,j,:]) == 1]

    # uniqueness in every column
    for j in digits:
        for k in digits:
            constraints += [np.sum(decisionVars[:,j,k]) == 1]
    
    # uniqueness in every row
    for i in digits:
        for k in digits:
            constraints += [np.sum(decisionVars[i,:,k]) == 1]

    # uniqueness in every sub-matrix
    for k in digits:
        for blockRow in submatrices:
            for blockCol in submatrices:
                constraint_expr = cp.sum([decisionVars[3*blockRow + i, 3*blockCol + j, k]
                                        for i in submatrices
                                        for j in submatrices])
                constraints.append(constraint_expr == 1)

    # initial board values 
    for i in digits:
        for j in digits:
            boardValue = board[i,j]
            if boardValue != 0:
                constraints.append(decisionVars[i, j, board[i,j] - 1] == 1)

    # building arbitrary objective function
    objectiveExpression = cp.sum([i*j*k*decisionVars[i,j,k] 
                                  for i in digits 
                                  for j in digits 
                                  for k in digits])
    objective = cp.Minimize(0)

    lp = cp.Problem(objective, constraints)
    lp.solve(solver="MOSEK", verbose=False)

    return decisionVars

def DecodeSolution(decisionVars:np.ndarray) -> np.ndarray:
    """ From a 3d array with binary variables representing a possible solution, build the corresponding 2d board.

    Args:
        decisionVars (np.ndarray): Binary decision variables representing a possible solution to a sudoku board.

    Returns:
        np.ndarray: Reconstructed sudoku board.
    """

    boardSize = len(decisionVars[0,:,:])
    digits = range(boardSize)

    solution = np.zeros((boardSize, boardSize), dtype=int)
    for i in digits:
        for j in digits:
            for k in digits:
                if decisionVars[i, j, k].value > 0.5:
                    solution[i, j] = k + 1
                    break
                
    return solution

def CheckSudoku(board):
    # Check if all rows contain digits 1-9 without duplicates
    for row in board:
        if not isValidGroup(row):
            return False

    # Check if all columns contain digits 1-9 without duplicates
    for col in range(9):
        column = [board[row][col] for row in range(9)]
        if not isValidGroup(column):
            return False

    # Check each 3x3 sub-box
    for box_row in range(3):
        for box_col in range(3):
            block = []
            for r in range(box_row * 3, box_row * 3 + 3):
                for c in range(box_col * 3, box_col * 3 + 3):
                    block.append(board[r][c])
            if not isValidGroup(block):
                return False

    return True

def isValidGroup(nums):
    # Check if the group contains digits 1-9 exactly once
    nums = [n for n in nums if n != 0]  # Ignore zeros if you want to allow incomplete boards
    return len(nums) == 9 and set(nums) == set(range(1, 10))

In [32]:
board = np.array([
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
])

solution = SolveSudoku(board)

solution = DecodeSolution(solution)
print(solution)
print("Is the board correctly filled: ", CheckSudoku(solution))

[[5 3 4 6 7 8 9 1 2]
 [6 7 2 1 9 5 3 4 8]
 [1 9 8 3 4 2 5 6 7]
 [8 5 9 7 6 1 4 2 3]
 [4 2 6 8 5 3 7 9 1]
 [7 1 3 9 2 4 8 5 6]
 [9 6 1 5 3 7 2 8 4]
 [2 8 7 4 1 9 6 3 5]
 [3 4 5 2 8 6 1 7 9]]
Is the board correctly filled:  True


## Questão 3: Senha

## Questão 4: 8 rainhas